In [ ]:
parent_dir = "C:/Users/Davide/git/gitdf/EnergyCommunity.jl/run_cloud/results_paper_ANC" #git/gitdf/EnergyCommunity.jl/run_cloud"
saveiter_dir = "$parent_dir/iter"
saveenum_dir = "$parent_dir/enum"

In [ ]:
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings
using YAML
using CSV
using Pipe

## Compare ENUM_MODE

In [ ]:
EC_size_list_enum = []
dict_enums = Dict()

for filename in readdir(saveenum_dir)
    if endswith(filename, ".jld2") && startswith(filename, "enum_simulations_results_")
        size_enum = parse(Int, replace(filename, "enum_simulations_results_"=>"", ".jld2"=>""))
        push!(EC_size_list_enum, size_enum)
        dict_enums[size_enum] = load("$saveenum_dir/enum_simulations_results_$size_enum.jld2")
    end
end

### ENUM_MODE: Create reward redistribution of enum modes

In [ ]:
df_reward_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_reward_temp = deepcopy(dict_enums[EC_size]["df_reward_enum"])
    df_reward_temp[!, "EC_size"] .= EC_size

    # Put users as columns
    df_reward_temp = unstack(stack(df_reward_temp), :user_set, :value)
    if nrow(df_reward_enum) == 0
        df_reward_enum = df_reward_temp
    else
        df_reward_enum = vcat(df_reward_enum, df_reward_temp, cols=:union)
    end
end

# sort by user set
df_reward_enum = df_reward_enum[!, ["variable"; "EC_size"; EC_CODE; ["user$i" for i = 1:(ncol(df_reward_enum)-3)]]]

df_reward_enum

### ENUM_MODE: Create comparison of computational time

In [ ]:
header_df_time = ["EC_size"]#, "title"]

df_time_enum = DataFrame()

for EC_size in EC_size_list_enum
    df_time_temp = DataFrame(dict_enums[EC_size]["df_time_enum"])  # dict_time_enum

    df_time_temp[!, setdiff(names(df_time_temp), ["name", "id_run", "EC_size"])] ./= 3600  # change units to hours

    # df_time_temp[!, "title"] = [L"Time [h]"]
    df_time_temp = df_time_temp[!, [header_df_time; setdiff(names(df_time_temp), header_df_time)]]

    if nrow(df_time_enum) == 0
        df_time_enum = df_time_temp
    else
        df_time_enum = vcat(df_time_enum, df_time_temp)
    end
end

df_time_enum

## Compare ITER_MODE

In [ ]:
df_run_simulations_iter = CSV.read("$saveiter_dir/options_backup.csv", DataFrame)

In [ ]:
dict_iter = Dict()
n_iter = 0

while isfile("$saveiter_dir/iter_simulations_results_$(n_iter+1).jld2")
    n_iter += 1
    dict_iter[n_iter] = load("$saveiter_dir/iter_simulations_results_$n_iter.jld2")
end

#### ITER_MODE: Comparison of reward distribution

In [ ]:
df_reward_iter = DataFrame()

largest_user_set = []

for id_run in 1:n_iter
    df_reward_temp = deepcopy(dict_iter[id_run]["df_reward_iter"])
    df_reward_temp[!, "EC_size"] .= nrow(df_reward_temp)-1
    df_reward_temp[!, "id_run"] .= id_run

    # Put users as columns
    df_reward_temp = unstack(stack(df_reward_temp), :user_set, :value)
    if nrow(df_reward_iter) == 0
        df_reward_iter = df_reward_temp
    else
        df_reward_iter = vcat(df_reward_iter, df_reward_temp, cols=:union)
    end
end

# sort by user set
df_reward_iter = df_reward_iter[!, ["variable"; "EC_size"; "id_run"; EC_CODE; ["user$i" for i = 1:(ncol(df_reward_iter)-4)]]]
# CSV.write("reward_iter.csv", df_reward_iter)
first(df_reward_iter, 5)

#### ITER_MODE: Create comparison of computational time

In [ ]:
df_time_iter = DataFrame()

for id_run in 1:n_iter
    df_time_temp = deepcopy(dict_iter[id_run]["df_time_iter"])

    df_time_temp[!, 3:end] ./= 3600  # change units to hours
    
    df_time_temp[!, "id_run"] .= id_run

    if nrow(df_time_iter) == 0
        df_time_iter = df_time_temp
    else
        df_time_iter = vcat(df_time_iter, df_time_temp)
    end
end

header_cols = ["name", "id_run", names(df_run_simulations_iter)...]
df_list_rewards = setdiff(names(df_time_iter), intersect(header_cols, names(df_run_simulations_iter)))

df_time_iter = hcat(df_run_simulations_iter[df_time_iter[!, "id_run"], :], df_time_iter[!, df_list_rewards])
df_time_iter = df_time_iter[!, [header_cols; setdiff(names(df_time_iter), header_cols)]]

# CSV.write("comp_time_iter.csv", df_time_iter)

df_time_iter

#### ITER_MODE: Create comparison of time per iteration

In [ ]:
df_history_iter = DataFrame()

for id_run in 1:n_iter
    df_history_temp = deepcopy(dict_iter[id_run]["df_history"])
    df_history_temp[!, "id_run"] .= id_run

    df_history_temp = df_history_temp[!, ["id_run"; setdiff(names(df_history_temp), ["id_run"])]]

    if nrow(df_history_iter) == 0
        df_history_iter = df_history_temp
    else
        df_history_iter = vcat(df_history_iter, df_history_temp)
    end
end
header_cols = ["name", "id_run"]
df_history_iter = df_history_iter[!, [header_cols; setdiff(names(df_history_iter), header_cols)]]

CSV.write("$parent_dir/raw_history.csv", df_history_iter)

first(df_history_iter, 5)

In [ ]:
# add elapsed time to history dataframe

sort!(df_history_iter, [:id_run, :name])

df_history_iter[!, :iteration_time] .= NaN

groups = groupby(df_history_iter, [:id_run, :name])

for (grp_key, grp) in zip(keys(groups), groups)
    grp[!, :iteration_time] = [grp[1, :elapsed_time]; diff(grp[!, :elapsed_time])]
end

In [ ]:
df_summary_iterations = @pipe df_history_iter |> groupby(_, [:name, :id_run]) |>
    combine(_, 
        :iteration => maximum => :max_iter,
        :elapsed_time => first => :initialization_time,
        :elapsed_time => maximum => :elapsed_time,
        :elapsed_time => (x -> maximum(x)/(length(x)-1)) => :mean_time,
        :iteration_time => maximum => :max_duration_iter,
        :iteration_time => (x -> sum(x .> 3 * 3600)) => :iteration_time_beyond_3h,
        )

first(df_summary_iterations, 5)

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot(title=k)

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        plot!(v_r[!, :iteration], v_r[!, :elapsed_time], label="id_run: $k_r")
    end

    xlabel!("Iteration [#]")
    ylabel!("Elapsed time [s]")

    display(p)
end

#### ITER_MODE: Comparison of iteration time

In [ ]:
for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    p = plot(title=k)

    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        delta_time = [v_r[1, :elapsed_time]; v_r[2:end, :elapsed_time] .- v_r[1:end-1, :elapsed_time]]
        plot!(v_r[!, :iteration], delta_time, label="id_run: $k_r")
    end

    xlabel!("Iteration [#]")
    ylabel!("Elapsed time [s]")

    display(p)
end

#### ITER_MODE: Comparison of upper and lower bounds

In [ ]:
rng_plot = 3

for ((k,), v) in pairs(groupby(df_history_iter, [:name]))
    # initialize plot
    # loop over run ids
    for ((k_r,), v_r) in pairs(groupby(v, [:id_run]))
        p = plot(v_r[!, :iteration], v_r[!, :value_min_surplus], label="Upper bound", color="red", title="$k - id_run: $k_r")
        plot!(v_r[!, :iteration], v_r[!, :lower_problem_min_surplus], label="Lower bound", color="blue")

        best_obj = minimum(v_r[!, :value_min_surplus])

        xlabel!("Iteration [#]")
        ylabel!("Obj. value")
        # ylims!(p, -rng_plot*best_obj, rng_plot*best_obj)
    
        display(p)
    end
end

## ALL: Merge all data into final files

#### Reward outputs

In [ ]:
header_df_reward = ["EC_size", "id_run", "variable"]

# create backup copies of dictionaries
df_reward_enum_copy = deepcopy(df_reward_enum)
df_reward_iter_copy = deepcopy(df_reward_iter)

# clean dataframes enum
df_reward_enum_copy[!, "name"] .= "enum"
df_reward_enum_copy[!, "id_run"] .= "-"
df_reward_enum_copy[!, "variable"] .= replace.(df_reward_enum_copy[!, "variable"], "_enum"=>"")

# clean dataframes iter
df_reward_iter_copy[!, "name"] .= "iter"
df_reward_iter_copy[!, "id_run"] = string.(df_reward_iter_copy[!, "id_run"])
df_reward_iter_copy[!, "variable"] .= replace.(df_reward_iter_copy[!, "variable"], "_iter"=>"")

# merge dataframes
df_reward_all = vcat(df_reward_enum_copy, df_reward_iter_copy, cols=:union)
df_reward_all = df_reward_all[!, [header_df_reward; setdiff(names(df_reward_iter), header_df_reward)]]

CSV.write("$parent_dir/reward.csv", df_reward_all)

first(df_reward_all, 5)

#### Computational time

In [ ]:
header_df_time = ["EC_size", "id_run"]

# work on copies of dataframes
df_time_enum_copy = df_time_enum[:, setdiff(names(df_time_enum), ["title", "id_run"])]
df_time_iter_copy = deepcopy(df_time_iter)

# clean dataframe enum
rename!(df_time_enum_copy, replace.(names(df_time_enum_copy), "_enum"=>""))
# rename!(df_time_enum_copy, :mode_time=>:mode)
df_time_enum_copy[!, "name"] = replace.(df_time_enum_copy[!, "name"], "_mode"=>"")

# clean dataframe iter
df_time_iter_copy[!, "name"] .= "iter"
rename!(df_time_iter_copy, replace.(names(df_time_iter_copy), "_iter"=>""))

# merge dataframes

df_time_all = vcat(df_time_enum_copy, df_time_iter_copy, cols=:union)
df_time_all = df_time_all[!, [header_df_time; setdiff(names(df_time_all), header_df_time)]]

CSV.write("$parent_dir/comp_time.csv", df_time_all)

df_time_all

#### Summary iterations

In [ ]:
df_run_simulations_iter_copy[!, :id_run]

In [ ]:
df_summary_iterations_copy = deepcopy(df_summary_iterations)
df_summary_iterations_copy[!, "name"] = replace.(df_summary_iterations_copy[!, "name"], "_iter"=>"")

df_run_simulations_iter_copy = deepcopy(df_run_simulations_iter)
df_run_simulations_iter_copy[!, :id_run] = 1:nrow(df_run_simulations_iter_copy)

df_summary_iterations_merged = leftjoin(df_summary_iterations_copy, df_run_simulations_iter_copy, on=[:id_run])

df_summary_iterations_merged = df_summary_iterations_merged[
    !, 
    [
        names(df_summary_iterations_copy);
        setdiff(names(df_summary_iterations_merged), names(df_summary_iterations_copy))
    ]
]

CSV.write("$parent_dir/summary_iterations.csv", df_summary_iterations_merged)

df_summary_iterations_merged